In [ ]:
import streamlit as st
import google.generativeai as genai
import json

# Initialize Gemini API
GOOGLE_API_KEY = 'your_gemini_api_key_here'
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-pro')

st.set_page_config(page_title="TalentScout AI Hiring Assistant", layout='centered')
st.title("🤖 TalentScout AI Hiring Assistant")

if 'candidate_data' not in st.session_state:
    st.session_state.candidate_data = {}
if 'questions' not in st.session_state:
    st.session_state.questions = {}
if 'step' not in st.session_state:
    st.session_state.step = 'greet'

exit_keywords = ['exit', 'quit', 'bye']

user_input = st.text_input("You:")

def collect_info_prompt(user_input):
    return f"""
You are an AI hiring assistant. Extract these candidate details from the input:
- Full Name
- Email Address
- Phone Number
- Years of Experience
- Desired Position
- Current Location
- Tech Stack (languages, frameworks, databases, tools)
Input: "{user_input}"
Respond ONLY in compact JSON.
"""

def generate_questions_prompt(tech_stack):
    return f"""
Generate 3-5 technical interview questions for each technology listed in:
{tech_stack}
Respond in JSON format as:
{{"Technology": "<name>", "Questions": ["Q1", "Q2", ...]}}
"""

if st.button("Send"):
    if any(k in user_input.lower() for k in exit_keywords):
        st.session_state.step = 'exit'

    if st.session_state.step == 'greet':
        st.write("Chatbot: Hello! Please provide your personal details and tech stack.")
        st.session_state.step = 'collect_info'

    elif st.session_state.step == 'collect_info':
        info_response = model.generate_content(collect_info_prompt(user_input))
        try:
            candidate_data = json.loads(info_response.text)
            st.session_state.candidate_data = candidate_data
            st.write("Chatbot: Thank you! Here is your provided information:")
            st.json(candidate_data)

            tech_stack = candidate_data.get('Tech Stack', '')
            if tech_stack:
                question_response = model.generate_content(generate_questions_prompt(tech_stack))
                questions = json.loads(question_response.text)
                st.session_state.questions = questions
                st.write("Chatbot: Here are your technical questions:")
                st.json(questions)
            else:
                st.write("Chatbot: Tech stack missing. Please provide your technologies.")

            st.session_state.step = 'completed'

        except json.JSONDecodeError:
            st.write("Chatbot: Unable to parse your details. Please re-enter in proper format.")

    elif st.session_state.step == 'completed':
        st.write("Chatbot: Thank you! Type 'exit' to end the conversation.")

    elif st.session_state.step == 'exit':
        st.write("Chatbot: Thank you for interacting! We will reach out to you soon.")

elif user_input:
    if st.session_state.step == 'greet':
        st.write("Chatbot: Welcome! Kindly provide your profile details and tech stack.")
